In [8]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/gdrive')
# Define base folder path
base_path = '/content/gdrive/MyDrive/RGB_data_stream'
data_path = '/content/gdrive/MyDrive/RGB_data_stream/data'
!ls /content/gdrive/MyDrive/RGB_data_stream/
!ls /content/gdrive/MyDrive/RGB_data_stream/data
# base_path = '../'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
1DCNNLSTM_Cross_fold
2DCNNLSTM-Jan28
CROSS_FOLD_2D_CNN_LSTM_2025-02-10_LR0.001_LHS512_LL1_CTResnet
CROSS_FOLD_2D_CNN_LSTM_2025-02-13_LR0.001_LHS512_LL1_CTResnet
CROSS_FOLD_2D_CNN_LSTM_2025-02-14_LR0.001_LHS512_LL1_CTResnet
data
gaussian_blur_aug
horz_aug
mobile_net_nov2
new_save
NEW_SHORTS
nov10_bi_lstm
nov10_equal_interval_sample
nov11_bi_lstm
nov11_Second_bi_lstm
nov12_unknown_attention
nov16_unknown_attention
nov7_equal_interval_sample
nov9_equal_interval_sample
pre_processed_short
resize_aug
RESIZED_SHORTS
Resnet_nov6
save
SECOND_mobile_net_nov2
Second_Resnet_nov6
short
SMPLer-X
SMPLR-X
VIBE
Without_augmentation_1DCNNLSTM_Cross_fold
2D_CNN_LSTM_folds.pkl	3DYoga90.csv	      criterion_fold_1.pkl  pose-index.csv
2D_CNN_LSTM_skf.pkl	3DYoga90.json	      folds.pkl		    skf.pkl
3DYoga90_corrected.csv	criterion_fold_0.pkl  mod-yoga-90-data.csv


In [ ]:
!apt-get install ffmpeg
!pip install pytubefix

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
import pandas
import numpy
import subprocess
import pytubefix as pytube
from pathlib import Path

# # Mount drive and set up directories
# drive.mount('/content/gdrive')
# base_path = '/content/gdrive/MyDrive/RGB_data_stream'
# output_dir = os.path.join(base_path, 'NEW_SHORTS')
# log_file = os.path.join(base_path, 'downloaded_log.txt')

# # Create output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# Helper function to find 720p video stream
def find_video_stream(url, retries=5):
    for _ in range(retries):
        try:
            yt = pytube.YouTube(url, use_po_token=True, allow_oauth_cache=True)
            yt.check_availability()
            streams = yt.streams.filter(resolution='720p', type='video')
            if streams:
                return streams[0]
        except Exception as e:
            print(f"Error accessing video {url}: {str(e)}")
            continue
    return None

def convert_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02}:{minutes:02}:{seconds:02}"

def log_download(sequence_id):
    with open(log_file, 'a') as f:
        f.write(f"{sequence_id}\n")

def get_downloaded_sequences():
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            return set(int(line.strip()) for line in f if line.strip())
    return set()

def clip_video(input_file, output_file, start_time, duration):
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', start_time,
        '-t', duration,
        '-c:v', 'libx264',
        '-c:a', 'aac',
        '-y',  # Overwrite output file if it exists
        output_file
    ]

    try:
        subprocess.run(command, check=True, capture_output=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg error: {e.stderr.decode()}")
        return False

def download_and_clip(sequence_id, data):
    output_file = os.path.join(output_dir, f"{sequence_id}.mp4")

    # Skip if already downloaded
    if os.path.exists(output_file):
        return True

    url = data['url']
    start_time = convert_seconds_to_hms(data['frame_start'])
    duration = convert_seconds_to_hms(data['sequence_duration'])

    # Get video stream
    stream = find_video_stream(url)
    if not stream:
        print(f"Could not find 720p stream for sequence {sequence_id}")
        return False

    try:
        # Download full video
        print(f"Downloading sequence {sequence_id}...")
        temp_file = stream.download()

        # Clip video
        print(f"Clipping sequence {sequence_id}...")
        if clip_video(temp_file, output_file, start_time, duration):
            # Clean up and log success
            os.remove(temp_file)
            log_download(sequence_id)
            return True
        else:
            print(f"Failed to clip video for sequence {sequence_id}")
            os.remove(temp_file)
            return False

    except Exception as e:
        print(f"Error processing sequence {sequence_id}: {str(e)}")
        if 'temp_file' in locals() and os.path.exists(temp_file):
            os.remove(temp_file)
        return False

def main():
    # Read the dataset
    data = pandas.read_csv(os.path.join(data_path, 'mod-yoga-90-data.csv')).set_index('sequence_id')

    # Define desired poses
    desired_poses = ['downward-dog','standing-forward-bend','half-way-lift',
             'mountain','chair','cobra','cockerel','extended-triangle',
             'extended-side-angle','corpse','staff','wind-relieving','fish'
            ]

    # Get sequences for desired poses
    sequences = data[data['l3_pose'].isin(desired_poses)].index.tolist()

    # Get already downloaded sequences
    downloaded = get_downloaded_sequences()

    # Process remaining sequences
    for seq_id in sequences:
        if seq_id in downloaded:
            print(f"Sequence {seq_id} already downloaded, skipping...")
            continue

        print(f"Processing sequence {seq_id}...")
        if download_and_clip(seq_id, data.loc[seq_id]):
            print(f"Successfully processed sequence {seq_id}")
        else:
            print(f"Failed to process sequence {seq_id}")

if __name__ == "__main__":
    main()

Processing sequence 1000...
Clipping sequence 1000...
Successfully processed sequence 1000
Processing sequence 1002...
Clipping sequence 1002...
Successfully processed sequence 1002
Processing sequence 1003...
Clipping sequence 1003...
Successfully processed sequence 1003
Processing sequence 1004...
Clipping sequence 1004...
Successfully processed sequence 1004
Processing sequence 1005...
Clipping sequence 1005...
Successfully processed sequence 1005
Processing sequence 1006...
Clipping sequence 1006...
Successfully processed sequence 1006
Processing sequence 1007...


In [ ]:
visitor = "CgtqUHhleTZQZkRmYyjojLy9BjIKCgJOUBIEGgAgGA%3D%3D"
po_token = "MnSrS3KbfBSaLatzpV5bns37pskOd84hrjFt0qMw0MOGNhmhrbGt6l775JDCeXbtT733J6QVnsk1lvSseOVv9Ftoa0qWsSNU58c06acqNhQi0j6XUMxHS0Oe9UrzPzxBbVH9_atU9vu_QMvKlG4xBqw9xQ1T2Q=="